# Imports

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import os
import wget
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor
import requests
import json
import os
import shutil

## Setting up webdriver

In [4]:
path_of_chromedriver = "D:/Softwares/chromedriver_win32/chromedriver.exe"

service = Service(path_of_chromedriver)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service , options=op)

link = "https://www.mov.onl/search/label/TV%20Series"
driver.get(link)
time.sleep(10)

### Scrolling page all the way down

In [5]:
scroll = True

while scroll==True:
    
    height = driver.execute_script("return document.body.scrollHeight")
    
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height==height:
        scroll=False



## Storing the link of each TV Show in LIST

In [6]:
html = driver.page_source
strainer = SoupStrainer('div', attrs={"id":"fold"})
soup = BeautifulSoup(html, 'html.parser', parse_only=strainer)


series_divs = soup.find_all('div', class_='story')

LINKS = []
SHOW_NAMES = []
for i in series_divs:
    link = i.find('h3', class_='title entry-title').a['href']
    name = i.find('h3', class_='title entry-title').a.text
    LINKS.append(link)
    SHOW_NAMES.append(name)
    
# driver.close()
print("Number of Shows:", len(LINKS))
print("Number of Shows Name:", len(SHOW_NAMES))

Number of Shows: 132
Number of Shows Name: 132


## Let's go to the each SHOW and retrieve download links of Episodes.

In [22]:
download_links_list = [] # nested list
episodes_name_list = [] # nested list

In [23]:
for i in range(len(LINKS)):
    
    EP = []          # episode list
    DWL = []         # download link list
    
    driver.get(LINKS[i])
    time.sleep(6)
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    
    linkss = soup2.find_all('source', type='video/mp4')
    ep_names = soup2.find_all('h2')
    
    for jj in ep_names:
        for nn in (jj.find_all('span')):
            Ep_name = nn.text
            if nn is None:
                EP.append("")
            else:
                EP.append(Ep_name)
    
    for j in range(len(linkss)):
        d_link = linkss[j]['src']
        DWL.append(d_link)
        
        
        
    episodes_name_list.append(EP)
    download_links_list.append(DWL)
    print('iteration:',i+1, "of", len(LINKS),"EPISODES: ", len(EP),",",len(DWL))

iteration: 1 of 132 EPISODES:  5 , 5
iteration: 2 of 132 EPISODES:  2 , 2
iteration: 3 of 132 EPISODES:  8 , 8
iteration: 4 of 132 EPISODES:  13 , 13
iteration: 5 of 132 EPISODES:  22 , 22
iteration: 6 of 132 EPISODES:  18 , 18
iteration: 7 of 132 EPISODES:  7 , 7
iteration: 8 of 132 EPISODES:  22 , 22
iteration: 9 of 132 EPISODES:  22 , 22
iteration: 10 of 132 EPISODES:  20 , 20
iteration: 11 of 132 EPISODES:  9 , 9
iteration: 12 of 132 EPISODES:  9 , 9
iteration: 13 of 132 EPISODES:  18 , 18
iteration: 14 of 132 EPISODES:  22 , 22
iteration: 15 of 132 EPISODES:  16 , 16
iteration: 16 of 132 EPISODES:  10 , 10
iteration: 17 of 132 EPISODES:  6 , 6
iteration: 18 of 132 EPISODES:  8 , 8
iteration: 19 of 132 EPISODES:  12 , 12
iteration: 20 of 132 EPISODES:  6 , 6
iteration: 21 of 132 EPISODES:  16 , 16
iteration: 22 of 132 EPISODES:  10 , 10
iteration: 23 of 132 EPISODES:  8 , 8
iteration: 24 of 132 EPISODES:  8 , 8
iteration: 25 of 132 EPISODES:  13 , 13
iteration: 26 of 132 EPISODES: 

In [24]:
# run when above loop completes.

driver.close()

In [31]:
for i in range(len(episodes_name_list)):
    if len(episodes_name_list[i])<len(download_links_list[i]):
        for j in range(len(episodes_name_list[i]),len(download_links_list[i])):
            episodes_name_list[i].append("")

#### Removing Special Characters from episodenames & shownames

In [32]:
for i in range(len(SHOW_NAMES)):
    SHOW_NAMES[i] = SHOW_NAMES[i].translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
    for j in range(len(episodes_name_list[i])):
        episodes_name_list[i][j] = episodes_name_list[i][j].translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
        

## Make Folders

In [33]:
def folders():
    for i in range(len(SHOW_NAMES)):
        SHOW_NAMES[i]= SHOW_NAMES[i].replace(':',"")
        SHOW_NAMES[i]= SHOW_NAMES[i].replace('?',"")
        path_ = str(SHOW_NAMES[i])
        os.mkdir(path_)
        

#### Run this cell to create directories.

In [ ]:
folders()

### Create flat lists of LINKS and EPISODE NAMES

In [34]:
from collections import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

In [35]:
newlinks = list(flatten(download_links_list))
print(len(newlinks))


1376


In [36]:
newepisodes = list(flatten(episodes_name_list))
print(len(newepisodes))

1376


##### the above numbers must be equal

In [ ]:
def getData(url:str):
    indx = newlinks.index(url)
    fname = newepisodes[indx]
    
    for i in range(len(episodes_name_list)):
        if fname in episodes_name_list[i]:
            FOLDER = SHOW_NAMES[i]
            ep_num = episodes_name_list[i].index(fname)
    
    show_file = f"{FOLDER}/{ep_num}_{fname}.mkv"
    print(f"Downloading......{fname}")
    with requests.get(url=url, allow_redirects=True, stream=True, timeout=30) as r:
        with open(show_file, 'wb') as writefile:
            shutil.copyfileobj(r.raw, writefile)
    print('Downloaded','\n')    

# Run the below cell to begin the show.

In [ ]:
#Multi-threading
with ThreadPoolExecutor(max_workers=1) as executor:
    executor.map(getData, newlinks)